In [1]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
#from scipy.stats import norm
# from scipy import stats
# from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [2]:
non_coding_region= "acceptor"
cancer_type= "Brain"

In [3]:
base_dir = "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/DNABERT_Data/{}/Patient_wise".format(cancer_type, non_coding_region)
output_dir= "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Variant_Analysis/{}/".format(cancer_type, non_coding_region)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
df_acceptor = pd.read_csv("/home/pdutta/Data/Noncoding_region/Unique_{}_RAW_Combined_BED.tsv".format(non_coding_region), sep="\t")
df_acceptor

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,12572,12651,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001758273.2, ENSE00003582793.1","transcribed_unprocessed_pseudogene, processed_..."
1,chr1,12934,13013,+,ENST00000450305.2,ENSE00001799933.2,transcribed_unprocessed_pseudogene
2,chr1,13180,13259,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001746346.2, ENSE00002312635.1","transcribed_unprocessed_pseudogene, processed_..."
3,chr1,13412,13491,+,ENST00000450305.2,ENSE00001863096.1,transcribed_unprocessed_pseudogene
4,chr1,14999,15078,-,ENST00000488147.1,ENSE00001935574.1,unprocessed_pseudogene
...,...,...,...,...,...,...,...
295391,chrY,57211720,57211799,+,"ENST00000340131.12_PAR_Y, ENST00000359512.8_PA...","ENSE00001956598.1, ENSE00001416295.3, ENSE0000...","retained_intron, protein_coding, processed_tra..."
295392,chrY,57213086,57213165,-,ENST00000507418.6_PAR_Y,ENSE00002023900.1,unprocessed_pseudogene
295393,chrY,57213318,57213397,-,ENST00000507418.6_PAR_Y,ENSE00002036959.1,unprocessed_pseudogene
295394,chrY,57213563,57213642,-,ENST00000507418.6_PAR_Y,ENSE00002021169.1,unprocessed_pseudogene


In [5]:
with open("/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/Intersected_Data/intersected_vcf_{}_data.pkl".format(non_coding_region), "rb") as file:
    loaded_dictionary = pickle.load(file)

In [6]:
len(loaded_dictionary)

432

In [7]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [8]:
patient_dfs= []
patient_id = []
for dirpath, dirnames, filenames in os.walk(base_dir):
    print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        print(dir_name)
        #input()
        df_temp = loaded_dictionary[dir_name]
        print(len(df_temp))
        df_temp = df_temp.drop_duplicates(subset=['chr_name', 'start', 'end', 'START_POS', 'END_POS', 'REF','ALT']).reset_index()
        concat_probab = np.load(dir_path+"/Prediction_result/pred_results.npy")
        print(len(concat_probab))
        print(len(df_temp))
        if(2*len(df_temp)==len(concat_probab)):
            #input()
            Reference_probab = concat_probab[::2]
            Alternative_probab = concat_probab[1::2]
            df_temp['Ref_probab']=Reference_probab
            df_temp['Alt_probab']=Alternative_probab
            df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
            df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
            #display(df_temp)
            #input()
            # Apply the filter conditions
            df_temp = df_temp[((df_temp['Ref_probab'] > 0.5) & (df_temp['Alt_probab'] <= 0.5)) | 
                      ((df_temp['Ref_probab'] <= 0.5) & (df_temp['Alt_probab'] > 0.5))]
            #display(df_temp)
            #input()
            patient_dfs.append(df_temp)
        else:
            print(dir_name)
            input()
        #input()
    break

/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/acceptor/Patient_wise ['a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_raw_pindel', '0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_raw_pindel', '3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan', 'af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_raw_pindel', '67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan', '447a08c8-3488-45d1-9262-23c42341296f_sanger_raw_pindel', '033db366-1af1-4bd9-8339-c54800b1de57_sanger_raw_pindel', 'f15d872a-7e31-4111-9d92-5ac813a8bc4f_sanger_raw_pindel', 'c6b9e176-a36b-4eb2-aa15-6a0155c1aeb5_CaVEMan', 'ef6c3ecc-0bb6-4035-9c45-57740f5bcaa1_sanger_raw_pindel', '3b89e48e-3e35-402f-b7f0-6e773839de4e_sanger_raw_pindel', '4132ab70-457f-41a9-aae3-340f9fe109e2_sanger_raw_pindel', 'd214490c-9f1c-4228-8cbc-7c8238942041_CaVEMan', '4e285b95-27ba-49b9-980a-6dbfc09a62c0_CaVEMan', '410db725-9a2f-4df3-8ace-6eedd4988cec_CaVEMan', 'b845bff2-f0eb-4375-95f0-0ebecab03c29_CaVEMan', '535e0e9e-9757-45b3-aafd-2b80908de134_CaVE

In [9]:
patient_dfs[0]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
24,29,chr1,11988386,11988465,+,"ENST00000674548.1, ENST00000675919.1",ENSE00003899401.1,protein_coding,chr1,11988402,...,I,11988421,16,11988403,16,661.5709838867188,0.382522,0.863425,0.415223,-3.351219
33,38,chr1,16041210,16041289,+,"ENST00000683578.1, ENST00000684324.1",ENSE00003919933.1,protein_coding,chr1,16041214,...,I,16041218,0,16041215,165,1931.72998046875,0.999752,0.072965,-0.926558,15.646363
44,62,chr1,17334283,17334362,+,ENST00000375453.5,ENSE00001467132.1,protein_coding,chr1,17334306,...,I,17334321,12,17334307,84,1428.3399658203125,0.054167,0.959508,0.868682,-8.692679
45,63,chr1,17334283,17334362,+,ENST00000375453.5,ENSE00001467132.1,protein_coding,chr1,17334306,...,I,17334321,12,17334307,24,745.094970703125,0.054167,0.753625,0.527129,-5.739093
100,122,chr1,47225850,47225929,-,"ENST00000294339.3, ENST00000371884.6",ENSE00001064461.1,protein_coding,chr1,47225895,...,D,47225911,3,47225896,20,593.0980224609375,0.019207,0.532159,0.272972,-5.860069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4650,5426,chr9,128515488,128515567,+,"ENST00000309971.9, ENST00000372770.4, ENST0000...","ENSE00003488363.1, ENSE00003919815.1, ENSE0000...","protein_coding, processed_transcript, nonsense...",chr9,128515489,...,D,128515501,1,128515490,54,1101.22998046875,0.999993,0.000682,-0.999304,27.658417
4651,5427,chr9,128522627,128522706,+,"ENST00000309971.9, ENST00000372770.4, ENST0000...","ENSE00000806966.1, ENSE00003916225.1","protein_coding, processed_transcript, nonsense...",chr9,128522634,...,D,128522659,23,128522635,40,930.5570068359375,0.986588,0.004019,-0.969391,14.153883
4657,5435,chr9,131080274,131080353,+,ENST00000679272.1,ENSE00003907890.1,protein_coding,chr9,131080281,...,D,131080308,25,131080282,5,337.1260070800781,0.999322,0.001219,-0.997427,20.204182
4691,5477,chrX,33941971,33942050,+,ENST00000439992.6,ENSE00001794028.2,lncRNA,chrX,33941971,...,D,33941984,1,33941972,30,766.2349853515625,0.999886,0.014682,-0.985090,19.161382


In [10]:
# Concatenate the patient dataframes and drop duplicates to get all unique regions across the patients
unique_patient_regions = pd.concat(patient_dfs).drop_duplicates(subset=['chr_name', 'transcript_id', 'start', 'end', 'START_POS', 'END_POS', 'REF', 'ALT'])
unique_patient_regions

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
24,29,chr1,11988386,11988465,+,"ENST00000674548.1, ENST00000675919.1",ENSE00003899401.1,protein_coding,chr1,11988402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,38,chr1,16041210,16041289,+,"ENST00000683578.1, ENST00000684324.1",ENSE00003919933.1,protein_coding,chr1,16041214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,62,chr1,17334283,17334362,+,ENST00000375453.5,ENSE00001467132.1,protein_coding,chr1,17334306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,63,chr1,17334283,17334362,+,ENST00000375453.5,ENSE00001467132.1,protein_coding,chr1,17334306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,122,chr1,47225850,47225929,-,"ENST00000294339.3, ENST00000371884.6",ENSE00001064461.1,protein_coding,chr1,47225895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,1681,chr15,50930764,50930843,+,"ENST00000261842.10, ENST00000558439.5, ENST000...",ENSE00000688560.1,"protein_coding, nonsense_mediated_decay",chr15,50930781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3705,4294,chr6,42619427,42619506,+,ENST00000372903.6,ENSE00001458959.2,protein_coding,chr6,42619448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,236,chr16,35244229,35244308,+,ENST00000657477.1,ENSE00003869713.1,lncRNA,chr16,35244269,...,0.0,160.0,.,8.700000e-10,1.00,GG/CCCGG,False,0.043000,GG/CCGGG,0.96
538,538,chr22,24627811,24627890,+,"ENST00000400380.5, ENST00000400382.6, ENST0000...","ENSE00003847263.1, ENSE00003632874.1, ENSE0000...","protein_coding, nonsense_mediated_decay, proce...",chr22,24627822,...,0.0,111.0,.,3.100000e-06,1.00,GG/GGGGG,False,0.000097,GG/AGGGG,1.00


In [11]:
filtered_regions = unique_patient_regions.groupby(['chr_name', 'transcript_id', 'start', 'end']).filter(lambda x: len(x) > 1 and not all(x['START_POS'].eq(x['START_POS'].iloc[0]) & x['END_POS'].eq(x['END_POS'].iloc[0]) & x['REF'].eq(x['REF'].iloc[0]) & x['ALT'].eq(x['ALT'].iloc[0])))
filtered_regions

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
24,29,chr1,11988386,11988465,+,"ENST00000674548.1, ENST00000675919.1",ENSE00003899401.1,protein_coding,chr1,11988402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,62,chr1,17334283,17334362,+,ENST00000375453.5,ENSE00001467132.1,protein_coding,chr1,17334306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,63,chr1,17334283,17334362,+,ENST00000375453.5,ENSE00001467132.1,protein_coding,chr1,17334306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,126,chr1,54227121,54227200,-,"ENST00000326956.12, ENST00000357475.9, ENST000...","ENSE00002162045.3, ENSE00003718240.2, ENSE0000...","processed_transcript, protein_coding, retained...",chr1,54227180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,130,chr1,54977823,54977902,-,ENST00000641734.1,ENSE00003812596.1,lncRNA,chr1,54977864,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4865,5395,chr8,126848479,126848558,+,ENST00000643428.1,ENSE00003824801.1,lncRNA,chr8,126848485,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4926,5462,chr9,15888942,15889021,+,ENST00000472806.1,ENSE00001842737.1,processed_transcript,chr9,15888948,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001,5541,chr9,77205272,77205351,+,"ENST00000360280.8, ENST00000376636.7, ENST0000...",ENSE00001171927.1,protein_coding,chr9,77205285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3705,4294,chr6,42619427,42619506,+,ENST00000372903.6,ENSE00001458959.2,protein_coding,chr6,42619448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pd.set_option('display.max_columns', None)
filtered_regions[filtered_regions['start']==11988386]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,END_POS,ID,REF,ALT,QUAL,FILTER,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,ASMD,ASRD,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
24,29,chr1,11988386,11988465,+,"ENST00000674548.1, ENST00000675919.1",ENSE00003899401.1,protein_coding,chr1,11988402,11988403,215d1338-e46a-11ea-bf47-e855f95b0627,G,GTT,480.0,FF006,True,2.0,.,I,11988421.0,16.0,11988403.0,16.0,661.5709838867188,0.382522,0.863425,0.415223,-3.351219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11,chr1,11988386,11988465,+,"ENST00000674548.1, ENST00000675919.1",ENSE00003899401.1,protein_coding,chr1,11988402,11988403,13237ade-5bd7-11eb-a8c8-4f558c91d660,G,T,.,MNP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.382522,0.942157,0.527264,-4.716601,130.5,0.86,0.0,87.0,.,0.0055,0.99,GT/GGGGG,False,0.0051,GG/GGGGT,0.99


In [14]:
unique_patient_regions[unique_patient_regions['start']==11988386]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,END_POS,ID,REF,ALT,QUAL,FILTER,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,ASMD,ASRD,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
24,29,chr1,11988386,11988465,+,"ENST00000674548.1, ENST00000675919.1",ENSE00003899401.1,protein_coding,chr1,11988402,11988403,215d1338-e46a-11ea-bf47-e855f95b0627,G,GTT,480.0,FF006,True,2.0,.,I,11988421.0,16.0,11988403.0,16.0,661.5709838867188,0.382522,0.863425,0.415223,-3.351219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11,chr1,11988386,11988465,+,"ENST00000674548.1, ENST00000675919.1",ENSE00003899401.1,protein_coding,chr1,11988402,11988403,13237ade-5bd7-11eb-a8c8-4f558c91d660,G,T,.,MNP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.382522,0.942157,0.527264,-4.716601,130.5,0.86,0.0,87.0,.,0.0055,0.99,GT/GGGGG,False,0.0051,GG/GGGGT,0.99


In [15]:
# Reset the option to its default (typically 20)
pd.reset_option('display.max_columns')

In [16]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates
# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {}, {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []
    data_score_change[key] = []

# Loop through each patient and for each region, get the Alt_probab score for that region for that patient
for df in patient_dfs:
    print(df.shape)
    #print(df)
    for _, region in unique_patient_regions.iterrows():
        #print(region)
        #input()
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr_name'] == region.chr_name) & (df['start'] == region.start)& (df['end'] == region.end) & (df['START_POS'] == region.START_POS) & (df['END_POS'] == region.END_POS) & (df['REF'] == region.REF) & (df['ALT'] == region.ALT) , score_column]
            #print(score_series)
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
                #print(score_series , score)
                #input()
            data[f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'].append(score)

(255, 29)
(238, 29)
(30, 32)
(270, 29)
(1, 32)
(258, 29)
(261, 29)
(254, 29)
(5, 32)
(362, 29)
(259, 29)
(268, 29)
(4, 32)
(3, 32)
(25, 32)
(8, 32)
(10, 32)
(5, 32)
(266, 29)
(280, 29)
(2, 32)
(310, 29)
(218, 29)
(1, 32)
(6, 32)
(248, 29)
(4, 32)
(2, 32)
(557, 29)
(22, 32)
(5, 32)
(272, 29)
(4, 32)
(271, 29)
(250, 29)
(255, 29)
(257, 29)
(5, 32)
(253, 29)
(3, 32)
(8, 32)
(261, 29)
(249, 29)
(1, 32)
(2, 32)
(4, 32)
(275, 29)
(24, 32)
(1, 32)
(1, 32)
(237, 29)
(7, 32)
(255, 29)
(298, 29)
(1, 32)
(279, 29)
(260, 29)
(33, 32)
(4, 32)
(11, 32)
(30, 32)
(271, 29)
(3, 32)
(2, 32)
(259, 29)
(8, 32)
(5, 32)
(247, 29)
(254, 29)
(3, 32)
(255, 29)
(0, 32)
(257, 29)
(275, 29)
(261, 29)
(248, 29)
(262, 29)
(243, 29)
(8, 32)
(6, 32)
(5, 32)
(294, 29)
(10, 32)
(271, 29)
(3, 32)
(2, 32)
(0, 32)
(243, 29)
(267, 29)
(35, 32)
(273, 29)
(247, 29)
(4, 32)
(275, 29)
(274, 29)
(4, 32)
(270, 29)
(255, 29)
(7, 32)
(254, 29)
(269, 29)
(244, 29)
(256, 29)
(257, 29)
(6, 32)
(23, 32)
(3, 32)
(6, 32)
(18, 32)
(245, 

In [17]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

,Patient_ID,chr1_11988386_11988465_11988402_11988403_G_GTT,chr1_16041210_16041289_16041214_16041215_G_GCATGAGC,chr1_17334283_17334362_17334306_17334307_A_AT,chr1_17334283_17334362_17334306_17334307_A_ATT,chr1_47225850_47225929_47225895_47225900_GCAGA_G,chr1_54204977_54205056_54204984_54204988_CCTT_C,chr1_54227121_54227200_54227180_54227181_A_AGG,chr1_54977823_54977902_54977864_54977865_G_GAAA,chr1_59021086_59021165_59021103_59021104_A_AT,...,chr8_126848479_126848558_126848485_126848488_CAA_C,chr9_15888942_15889021_15888948_15888952_CTTT_C,chr9_77205272_77205351_77205285_77205287_CT_C,chr9_83855015_83855094_83855038_83855041_CTT_C,chrX_118447019_118447098_118447031_118447034_ATT_A,chr15_50930764_50930843_50930781_50930782_G_GT,chr6_42619427_42619506_42619448_42619457_ATATATTTT_A,chr16_35244229_35244308_35244269_35244270_G_C,chr22_24627811_24627890_24627822_24627823_G_A,chr6_32522135_32522214_32522156_32522157_C_A
0,a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_ra...,0.382522,0.999752,0.054167,0.054167,0.019207,0.999959,0.260495,0.534012,0.999607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,NaN,0.999752,NaN,0.054167,0.019207,NaN,0.260495,0.534012,0.999607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_ra...,NaN,0.999752,NaN,0.054167,0.019207,NaN,0.260495,0.534012,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,b0218338-6d83-491f-b51d-844d209f54b3_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,96a823e7-9a5c-4a58-a424-a051cf4f4b29_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,8db95363-6eb3-4a90-affd-0c948a05c27c_sanger_ra...,NaN,0.999752,NaN,0.054167,0.019207,NaN,NaN,0.534012,NaN,...,0.999517,0.999847,0.999716,0.999341,0.99992,NaN,NaN,NaN,NaN,NaN
430,cf565879-9f53-4fd1-b0da-065869293342_sanger_ra...,NaN,0.999752,NaN,NaN,NaN,NaN,NaN,NaN,0.999607,...,NaN,NaN,NaN,NaN,NaN,0.987475,0.992459,NaN,NaN,NaN


In [18]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

,Patient_ID,chr1_11988386_11988465_11988402_11988403_G_GTT,chr1_16041210_16041289_16041214_16041215_G_GCATGAGC,chr1_17334283_17334362_17334306_17334307_A_AT,chr1_17334283_17334362_17334306_17334307_A_ATT,chr1_47225850_47225929_47225895_47225900_GCAGA_G,chr1_54204977_54205056_54204984_54204988_CCTT_C,chr1_54227121_54227200_54227180_54227181_A_AGG,chr1_54977823_54977902_54977864_54977865_G_GAAA,chr1_59021086_59021165_59021103_59021104_A_AT,...,chr8_126848479_126848558_126848485_126848488_CAA_C,chr9_15888942_15889021_15888948_15888952_CTTT_C,chr9_77205272_77205351_77205285_77205287_CT_C,chr9_83855015_83855094_83855038_83855041_CTT_C,chrX_118447019_118447098_118447031_118447034_ATT_A,chr15_50930764_50930843_50930781_50930782_G_GT,chr6_42619427_42619506_42619448_42619457_ATATATTTT_A,chr16_35244229_35244308_35244269_35244270_G_C,chr22_24627811_24627890_24627822_24627823_G_A,chr6_32522135_32522214_32522156_32522157_C_A
0,a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_ra...,0.863425,0.072965,0.959508,0.753625,0.532159,0.000011,0.555806,0.012205,0.000756,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,NaN,0.072965,NaN,0.753625,0.532159,NaN,0.555806,0.012205,0.000756,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_ra...,NaN,0.072965,NaN,0.753625,0.532159,NaN,0.555806,0.012205,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,b0218338-6d83-491f-b51d-844d209f54b3_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,96a823e7-9a5c-4a58-a424-a051cf4f4b29_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,8db95363-6eb3-4a90-affd-0c948a05c27c_sanger_ra...,NaN,0.072965,NaN,0.753625,0.532159,NaN,NaN,0.012205,NaN,...,0.000024,0.000036,0.000204,0.011584,0.338172,NaN,NaN,NaN,NaN,NaN
430,cf565879-9f53-4fd1-b0da-065869293342_sanger_ra...,NaN,0.072965,NaN,NaN,NaN,NaN,NaN,NaN,0.000756,...,NaN,NaN,NaN,NaN,NaN,0.140479,0.002575,NaN,NaN,NaN


In [19]:
result_df_score_change = pd.DataFrame(data_score_change)
result_df_score_change.insert(0, "Patient_ID", patient_id)
result_df_score_change

,Patient_ID,chr1_11988386_11988465_11988402_11988403_G_GTT,chr1_16041210_16041289_16041214_16041215_G_GCATGAGC,chr1_17334283_17334362_17334306_17334307_A_AT,chr1_17334283_17334362_17334306_17334307_A_ATT,chr1_47225850_47225929_47225895_47225900_GCAGA_G,chr1_54204977_54205056_54204984_54204988_CCTT_C,chr1_54227121_54227200_54227180_54227181_A_AGG,chr1_54977823_54977902_54977864_54977865_G_GAAA,chr1_59021086_59021165_59021103_59021104_A_AT,...,chr8_126848479_126848558_126848485_126848488_CAA_C,chr9_15888942_15889021_15888948_15888952_CTTT_C,chr9_77205272_77205351_77205285_77205287_CT_C,chr9_83855015_83855094_83855038_83855041_CTT_C,chrX_118447019_118447098_118447031_118447034_ATT_A,chr15_50930764_50930843_50930781_50930782_G_GT,chr6_42619427_42619506_42619448_42619457_ATATATTTT_A,chr16_35244229_35244308_35244269_35244270_G_C,chr22_24627811_24627890_24627822_24627823_G_A,chr6_32522135_32522214_32522156_32522157_C_A
0,a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_ra...,0.415223,-0.926558,0.868682,0.527129,0.272972,-0.999906,0.164135,-0.278651,-0.998459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,NaN,-0.926558,NaN,0.527129,0.272972,NaN,0.164135,-0.278651,-0.998459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_ra...,NaN,-0.926558,NaN,0.527129,0.272972,NaN,0.164135,-0.278651,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,b0218338-6d83-491f-b51d-844d209f54b3_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,96a823e7-9a5c-4a58-a424-a051cf4f4b29_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,8db95363-6eb3-4a90-affd-0c948a05c27c_sanger_ra...,NaN,-0.926558,NaN,0.527129,0.272972,NaN,NaN,-0.278651,NaN,...,-0.99901,-0.999658,-0.999228,-0.987106,-0.661694,NaN,NaN,NaN,NaN,NaN
430,cf565879-9f53-4fd1-b0da-065869293342_sanger_ra...,NaN,-0.926558,NaN,NaN,NaN,NaN,NaN,NaN,-0.998459,...,NaN,NaN,NaN,NaN,NaN,-0.836386,-0.982419,NaN,NaN,NaN


In [20]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

,Patient_ID,chr1_11988386_11988465_11988402_11988403_G_GTT,chr1_16041210_16041289_16041214_16041215_G_GCATGAGC,chr1_17334283_17334362_17334306_17334307_A_AT,chr1_17334283_17334362_17334306_17334307_A_ATT,chr1_47225850_47225929_47225895_47225900_GCAGA_G,chr1_54204977_54205056_54204984_54204988_CCTT_C,chr1_54227121_54227200_54227180_54227181_A_AGG,chr1_54977823_54977902_54977864_54977865_G_GAAA,chr1_59021086_59021165_59021103_59021104_A_AT,...,chr8_126848479_126848558_126848485_126848488_CAA_C,chr9_15888942_15889021_15888948_15888952_CTTT_C,chr9_77205272_77205351_77205285_77205287_CT_C,chr9_83855015_83855094_83855038_83855041_CTT_C,chrX_118447019_118447098_118447031_118447034_ATT_A,chr15_50930764_50930843_50930781_50930782_G_GT,chr6_42619427_42619506_42619448_42619457_ATATATTTT_A,chr16_35244229_35244308_35244269_35244270_G_C,chr22_24627811_24627890_24627822_24627823_G_A,chr6_32522135_32522214_32522156_32522157_C_A
0,a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_ra...,-3.351219,15.646363,-8.692679,-5.739093,-5.860069,31.012842,-1.828692,6.535246,21.681342,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,NaN,15.646363,NaN,-5.739093,-5.860069,NaN,-1.828692,6.535246,21.681342,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_ra...,NaN,15.646363,NaN,-5.739093,-5.860069,NaN,-1.828692,6.535246,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,b0218338-6d83-491f-b51d-844d209f54b3_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,96a823e7-9a5c-4a58-a424-a051cf4f4b29_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,8db95363-6eb3-4a90-affd-0c948a05c27c_sanger_ra...,NaN,15.646363,NaN,-5.739093,-5.860069,NaN,NaN,6.535246,NaN,...,26.347384,27.432106,24.039176,16.981796,14.571977,NaN,NaN,NaN,NaN,NaN
430,cf565879-9f53-4fd1-b0da-065869293342_sanger_ra...,NaN,15.646363,NaN,NaN,NaN,NaN,NaN,NaN,21.681342,...,NaN,NaN,NaN,NaN,NaN,8.914004,15.637641,NaN,NaN,NaN


In [21]:
result_df_ref.to_csv(output_dir+"df_ref_score.tsv", sep="\t", index=False)
result_df_alt.to_csv(output_dir+"df_alt_score.tsv", sep="\t", index=False)
result_df_log_odd.to_csv(output_dir+"df_log_odd_score.tsv", sep="\t", index=False)
result_df_score_change.to_csv(output_dir+"df_score_change.tsv", sep="\t", index=False)

In [ ]:
output_dir